### H3 is a Uber library for indexing Hexagonal Hierarchal spaces
We will ilustrate how to use it and try to cluster data using h3. Might not be usefull for the hackathon goal or for these dataset, since grid system are use for analyzing large datasets.<br>
**Maps will not display on github using folium**, please clone, uncomment necessary lines and run locally.<br>
H3 supports sixteen resolutions. Each finer resolution has cells with one seventh the area of the coarser resolution. 0 being coarsest and 15 being finest.<br>
Instead of KMeans you can try using H3 or a combination.

In [16]:
import h3
import pandas as pd
from folium import Map, Marker, GeoJson
from geojson.feature import *
import branca.colormap as cm
import json

In [2]:
path = "../ubicaciones.csv"

In [3]:
df = pd.read_csv(path)

In [4]:
df.head()

,Id_Cliente,id_Agencia,Frecuencia,Vol_Entrega,lat,lon
0,1,A1,1,2.0,20.506052,-98.212377
1,2,A1,1,17.0,20.458128,-98.221248
2,3,A1,1,2.0,20.530427,-98.236860
3,4,A1,3,15.0,20.488655,-98.202940
4,5,A1,1,2.0,20.533253,-98.219669


In [5]:
lat_lon = df[["lat", "lon"]].values

In [16]:
#smaller zoom start further away
map = Map(location=[20.506052, -98.212377], zoom_start=10, titles ="All points")
for point in range(0, len(lat_lon)):
    Marker(lat_lon[point], popup=df['Id_Cliente'][point]).add_to(map)
    
#map

In [6]:
#getting index ofparticular lat-lon
h3.geo_to_h3(20.506052, -98.212377, 7)

'874991c98ffffff'

In [7]:
#0 to 15, where the former is bigger space
df["hex"] = df.apply(lambda row: h3.geo_to_h3(row["lat"], row["lon"], 6), axis = 1)

In [8]:
aggregated_df = df.groupby(by = "hex").size().reset_index()
aggregated_df.columns = ["hex", "stores"]

In [9]:
aggregated_df["geometry"] =  aggregated_df.hex.apply(lambda x: {"type" : "Polygon",
                                                           "coordinates": [h3.h3_to_geo_boundary(h=x, geo_json=True)]})

In [10]:
aggregated_df

,hex,stores,geometry
0,864990247ffffff,14,"{'type': 'Polygon', 'coordinates': [((-98.0829..."
1,86499024fffffff,446,"{'type': 'Polygon', 'coordinates': [((-98.1193..."
2,864990267ffffff,60,"{'type': 'Polygon', 'coordinates': [((-98.1126..."
3,86499026fffffff,757,"{'type': 'Polygon', 'coordinates': [((-98.1490..."
4,864991507ffffff,6,"{'type': 'Polygon', 'coordinates': [((-98.2218..."
5,864991517ffffff,2,"{'type': 'Polygon', 'coordinates': [((-98.1557..."
6,864991527ffffff,126,"{'type': 'Polygon', 'coordinates': [((-98.2515..."
7,86499152fffffff,1,"{'type': 'Polygon', 'coordinates': [((-98.2880..."
8,864991537ffffff,515,"{'type': 'Polygon', 'coordinates': [((-98.1854..."
9,864991c87ffffff,243,"{'type': 'Polygon', 'coordinates': [((-98.2447..."


In [15]:
min_value = aggregated_df.stores.min()
max_value = aggregated_df.stores.max()
initial_map = Map(location= [20.506052, -98.212377], zoom_start=11, titles="choropleth")
custom_cm = cm.LinearColormap(['green','yellow','red'], vmin=min_value, vmax=max_value)

list_features = []
for i, row in aggregated_df.iterrows():
    feature = Feature(geometry = row["geometry"] , id=row["hex"], properties = {"value" : row["stores"]})
    list_features.append(feature)
        
feat_collection = FeatureCollection(list_features)    
geojson_data = json.dumps(feat_collection)

GeoJson(geojson_data, style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties']['value']),
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7}, 
        name = "Choropleth_6").add_to(initial_map)

In [19]:
#display(initial_map)